# Info bici

In [1]:
import pickle

import geopandas as gpd
import osmnx as ox

from my_paths import *
import Graph_functions

## Strade/Ciclabili

### Piste ciclabili *NON PIU' USATO

Prima fase di pulizia del geojson raw

In [3]:
gdf_ciclabili_raw = gpd.read_file(PATH_CICLABILI_RAW)
gdf_ciclabili_staging = gdf_ciclabili_raw.copy()

# Convertiamo alcuni tipi
gdf_ciclabili_staging["geometry"] = gdf_ciclabili_staging["geometry"].apply(lambda x: x.geoms[0])
gdf_ciclabili_staging["fine_lavor"] = gdf_ciclabili_staging["fine_lavor"].astype(int)
gdf_ciclabili_staging = gdf_ciclabili_staging.rename(columns={"anagrafica": "name", "rete": "highway"})
gdf_ciclabili_staging = gdf_ciclabili_staging.drop(["id_amat", "id_via","gerarchia", "lunghezza"], axis=1)

gdf_ciclabili_staging = gpd.GeoDataFrame(gdf_ciclabili_staging, geometry="geometry")

Creazione del grafo + archi di congiunzione tra nodi molto vicini (5 metri).
Aggiunta di alcuni attributi utili:
- **"tipo"** = Per identificare in futuro la provenienza degli archi.
- **"artificiale"** = "False" se riferiti a veri percorsi presi dal geojson, "True" se aggiunti per scopi di calcolo percorsi.
N.B: "geojson_to_graph()" manterrà all'interno degli archi del grafo anche tutti gli attributi delle colonne del gdf.

In [4]:
G_ciclabili = Graph_functions.geojson_to_graph(gdf_ciclabili_staging, weight_moltiplicator=0, tipo = "Ciclabile", artificiale = False).to_undirected()
G_ciclabili = Graph_functions.add_edge_near_nodes(G_ciclabili, weight_moltiplicator = 0, tipo = "Ciclabile", artificiale = True)

Salviamo grafo (in pickle) e geojson in staging

In [5]:
gdf_edges = ox.graph_to_gdfs(G_ciclabili, edges=True, nodes=False).reset_index().drop(["u","v", "key"], axis=1)
gdf_edges.to_file(PATH_CICLABILI_GEOJSON_STAGING, driver="GeoJSON") # Anche questo andrebbe ripulito
with open(PATH_CICLABILI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_ciclabili, f)

### Strade ciclabili

#### Ingestion (openstreet maps)

In [ ]:
place = "Milano, Italy"

bike_filter = (
    '["highway"]["bicycle"!~"no"]["bicycle"!~"dismount"]'
)
G_ox_strade = ox.graph_from_place(place, network_type='bike', custom_filter=bike_filter)
with open(PATH_STRADE_CICLABILI_PICKLE_RAW, "wb") as f:
    pickle.dump(G_ox_strade, f)

#### ETL 

Processo di ETL sul Grafo Raw scaricato da openstreet maps.
Il proceso comprende:
1. Selezione attributi da mantenere: name, highway, lenght, maxspeed, tunnel, access, service, geometry
2. - Aggiunta di attributi custom:
    - **weight_multipler**: Un moltiplicatore che verrà applicato a weigth in base a "highway" tramite
    il parametro "custom_weights", oppure da file se non specificato.
    N.B: di base cycleway dovrebbe assumere valore: 0, per dare massima priorità nel calcolo dei percorsi.
    - **weight**: Sarà di base: length * weight_multipler.
    - **tipo**: Un tag che viene assegnato a tutti i grafi del progetto. In questo caso assumerà valore "Strade_ciclabili".
    - **artificial**: Un tag che viene assegnato a tutit i grafi del progetto. Essendo questi archi reali: False
    - **poi**: Un tag che viene assegnato a tutti i grafi del progetto. Essendo queste le strade: False
    - **ciclabile**: Un tag booleano per filtrare le piste ciclabili dalle altre strade.
3. Se Geometria non presente, aggiungiamo una LineString "dritta" tra i due nodi
(può capitare che non ci sia tra nodi molto vicini).
4. Infine il grafo viene reso bidirezionale senza ridondanza di archi tra due punti.

In [ ]:
G_strade = Graph_functions.etl_strade_ciclabili()

Usiamo la nostra funzione per convertire il grafo in gdf, che oltre a convertire gli archi in geodataframe esegue:
1. **Eliminazione** delle eventuali **"liste"** di elementi rendendole delle stringhe concatenate.
2. Interpolazione con geometria dei Municipi (con eventuale spezzettamento delle LineString delle strade a cavallo su più Municipi), 
    e quindi **aggiunta** della colonna **"MUNICIPI"** per ogni strada (utile per join di visualizzazione)

In [ ]:
gdf_strade = Graph_functions._custom_graph_to_gdf(G_strade)

Salvataggio in staging, sia grafo che geojson.

In [ ]:
gdf_strade.to_file(PATH_STRADE_CICLABILI_GEOJSON_CLEAN, driver="GeoJSON")
with open(PATH_STRADE_CICLABILI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_strade, f)